In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/01'

PROB = True
# TH = 0.016
# TH = 0.018
TH = 0.020
# TH = 0.0201
# TH = 0.0205
# TH = 0.0207
# TH = 0.0135
# TH = 0.0195

BORDER = 0.023282372444280715



```
sub_sugi_UMLSBert_ENG_fold4_epoch5.csv    0.9177989 
sub_sugi_BiomedNLP-PubMedBERT_fold4.csv   0.9151481 
sub_sugi_UMLSBert_ENG_fold4.csv           0.9134154
sub_sugi_BiomedNLP-PubMedBERT.csv    0.9129580
sub_sugi_BiomedNLP-PubMedBERT_fold3.csv   0.9114425 
sub_sugi_UMLSBert_ENG.csv                 0.9091279 
sub_sugi_scibert_scivocab_uncased.csv     0.9054163
sub_sugi_bluebert.csv                     0.9051148 
sub_sugi_Bio_Discharge_Summary_BERT.csv   0.9032105 
sub_masa_BioRedditBERT-uncased.csv        0.8991509
sub_sugi_BioRedditBERT-uncased.csv   0.8985702 
sub_sugi_scibert_scivocab_cased.csv       0.8977792 
sub_sugi_COVID-SciBERT.csv                0.8949713
sub_masa_COVID-SciBERT.csv                0.8895940 
```



In [30]:
CSV_LIST = [
               'sub_prob_008',
               'sub_prob_015',
               'sub_prob_018',
               'sub_prob_311',
            #    'sub_sugi_BiomedNLP-PubMedBERT',
            #    'sub_sugi_BiomedNLP-PubMedBERT_fold3',
            #    'sub_sugi_UMLSBert_ENG',
            #    'sub_sugi_scibert_scivocab_uncased',
            #    'sub_sugi_bluebert',
            #    'sub_sugi_Bio_Discharge_Summary_BERT',
            #    'sub_masa_BioRedditBERT-uncased',
            #    'sub_sugi_BioRedditBERT-uncased',
            #    'sub_sugi_scibert_scivocab_cased',
            #    'sub_sugi_COVID-SciBERT',
            ]
WEIGHTS = [1,1,1,0.5]

def read_data(csv_name):
    f = '{}/{}.csv'.format(DATA_DIR, csv_name)
    if PROB:
        f = '{}/{}.csv'.format(DATA_DIR, csv_name)

    df_ = pd.read_csv(f, header=None, names=['id', 'prob'])
    # df_[df_.prob >= 0.01].prob.plot.hist(bins=50)
    # plt.show()

    return df_

df = pd.DataFrame()

i = 0
for c in CSV_LIST:
    print(c)
    if len(df) == 0:
        df = read_data(c)
        # print(df.prob.quantile(1-BORDER))
        # df.prob.plot.hist(bins=50)
        df.prob *= WEIGHTS[i]
    else:
        df_ = read_data(c)
        # print(df_.prob.quantile(1-BORDER))
        df_.prob *= WEIGHTS[i]
        df = pd.concat([df, df_['prob']], axis=1)

    i += 1

df.set_index('id', inplace=True)
df

sub_prob_008
sub_prob_015
sub_prob_018
sub_prob_311


,prob,prob,prob,prob
id,,,,
27145,0.000323,0.000397,0.000446,0.000252
27146,0.000749,0.001710,0.001374,0.000533
27147,0.000642,0.000728,0.001984,0.000674
27148,0.000652,0.000732,0.001175,0.000655
27149,0.000579,0.000601,0.000690,0.000373
...,...,...,...,...
67974,0.000722,0.000805,0.000895,0.000719
67975,0.000478,0.000560,0.000532,0.000341
67976,0.001709,0.003800,0.003905,0.002047


In [31]:
df['judgement'] = 0
df['judgement'] = df.sum(axis=1)

if PROB:
    # df['judgement'] = df.judgement / (len(CSV_LIST) * 1.0)
    df['judgement'] = df.judgement / (sum(WEIGHTS) * 1.0)
    df[['judgement']].to_csv('{}/sub_prob_503.csv'.format(DATA_DIR), header=False)
    df['judgement'] = np.where(df.judgement < TH, 0, 1)
else:
    # df['judgement'] = np.where(df.judgement < len(CSV_LIST) // 2, 0, 1)
    df['judgement'] = np.where(df.judgement < sum(WEIGHTS) // 2, 0, 1)

df.reset_index(inplace=True)
print(df.judgement.sum())
df.head(20)

3740


,id,prob,prob,prob,prob,judgement
0,27145,0.000323,0.000397,0.000446,0.000252,0
1,27146,0.000749,0.001710,0.001374,0.000533,0
2,27147,0.000642,0.000728,0.001984,0.000674,0
3,27148,0.000652,0.000732,0.001175,0.000655,0
4,27149,0.000579,0.000601,0.000690,0.000373,0
5,27150,0.000551,0.000589,0.001087,0.000468,0
6,27151,0.016225,0.009493,0.013092,0.012316,0
7,27152,0.002151,0.001441,0.002198,0.001205,0
8,27153,0.000832,0.000984,0.000793,0.000502,0
9,27154,0.006018,0.005170,0.010815,0.006208,0


In [32]:
df[['id', 'judgement']].to_csv('{}/sub_503.csv'.format(DATA_DIR), index=False, header=False)